# wo shi ji ji kun !!!
# wuda shi wo ge!!!

In [1]:
from lmfit import Parameters
from lmfit import minimize
import numpy as np
import pandas as pd

### Define parameters

In [2]:
# risk aversion
delta = 4  # 3.5 < delta < 4.5

In [3]:
# convariance_quantities
sigma_all = [[.02778, .00387, .00021],
             [.00387, .01112, -.00020],
             [.00021, -.00020, .00115],
            ]

In [4]:
pd.DataFrame(sigma_all)

,0,1,2
0,0.02778,0.00387,0.00021
1,0.00387,0.01112,-0.00020
2,0.00021,-0.00020,0.00115


In [5]:
# expect_return
expect_return = [0.1073, 0.0737, 0.0627]

### Generate intial guess for xi 

In [6]:
# Generate random xi or input yours as list
nbr_assets = len(expect_return)
xi_weights = list(np.random.dirichlet(np.ones(nbr_assets),size=1)[0]) # generate random xi list
if sum(xi_weights)!=1: # test the sum is 1
    raise ValueError("The sum of your input weight '{}' is not '1'.".format(xi_weights))
print(xi_weights)
print("Sum: {}".format(sum(xi_weights)))

[0.1248767861310295, 0.56475222319501195, 0.31037099067395857]
Sum: 1.0


### Form params for minimization

In [7]:
# pack parameters for fitting
params = Parameters()
for n in range(len(xi_weights)):
    params.add('x' + str(n+1),
                value=xi_weights[n],
                vary=True,
                min=0,
                max=1)
print("\n----------Before apply constraint---------- \n")
params.pretty_print()

# form constraint
_param_list = list(params.valuesdict().keys())
_constraint_param = _param_list.pop(-1)
_constraint = '-'.join(_param_list)
_constraint = '1-' + _constraint

# set constraint
params[_constraint_param].set(expr=_constraint)
print("\n----------After apply constraint---------- \n")
params.pretty_print()



----------Before apply constraint---------- 

Name     Value      Min      Max   Stderr     Vary     Expr Brute_Step
x1    0.1249        0        1     None     True     None     None
x2    0.5648        0        1     None     True     None     None
x3    0.3104        0        1     None     True     None     None

----------After apply constraint---------- 

Name     Value      Min      Max   Stderr     Vary     Expr Brute_Step
x1    0.1249        0        1     None     True     None     None
x2    0.5648        0        1     None     True     None     None
x3    0.3104        0        1     None    False  1-x1-x2     None


# Define model

In [8]:
def model(params, expect_return, sigma_all):
    # unpack params
    parvals = params.valuesdict()
    param_names = parvals.keys()
    xi_weights = []
    for each in param_names:
        xi_weights.append(parvals[each])
    print("Trying: {}".format(xi_weights))
    
    # calculate the cost
    sum_1 = 0
    sum_2 = 0
    for i in range(len(xi_weights)):
        j=0
        _sum_1 = xi_weights[i] * expect_return[i]
        sum_1 = sum_1 + _sum_1
#         print(i)
        while j <= len(xi_weights)-1:
#             print(i, j)
            _sum_2 = sigma_all[i][j] * xi_weights[i] * xi_weights[j]
            sum_2 = sum_2 + _sum_2
            j=j+1
    cost = -(sum_1 - (delta/2)*sum_2)
    print(-cost)
    return cost
    

In [9]:
model(params, expect_return=expect_return, sigma_all=sigma_all)

Trying: [0.1248767861310295, 0.56475222319501195, 0.31037099067395857]
0.0653164178911


-0.065316417891058201

In [10]:
def jacobian(x):
    return np.array((-2*.5*(1 - x[0]) - 4*x[0]*(x[1] - x[0]**2), 2*(x[1] - x[0]**2)))

In [11]:
result = minimize(model, params, method='newton', jac=jacobian, args=(expect_return, sigma_all))

Trying: [0.12487678613102948, 0.56475222319501195, 0.31037099067395857]
0.0653164178911
Trying: [0.26977714009712866, 0.62041856152351404, 0.10980429837935723]
0.066363345901
Trying: [0.27041212380573937, 0.61974796677040667, 0.10983990942385402]
0.0663803105608
Trying: [0.27295674124508063, 0.61706331470334841, 0.10997994405157097]
0.0664477211738
Trying: [0.28320867149596723, 0.60628977425720465, 0.11050155424682817]
0.0667100512375
Trying: [0.32529686926726831, 0.56272923078488046, 0.11197389994785123]
0.0676336967859
Trying: [0.5045861629456565, 0.38703056835370409, 0.10838326870063941]
0.0689220228294
Trying: [0.5067502279166981, 0.38800927567151366, 0.10524049641178823]
0.0688719889114
Trying: [0.50503899860079327, 0.38723532373532277, 0.10772567766388397]
0.0689116088923
Trying: [0.50468068222870599, 0.38707330465233614, 0.10824601311895787]
0.0689198515911
Trying: [0.50460588132636097, 0.38703948381742259, 0.10835463485621644]
0.0689215699772
Trying: [0.50459027609281659, 0.387

In [12]:
result.params

Parameters([('x1', <Parameter 'x1', 0.5045861629456565, bounds=[0:1]>),
            ('x2', <Parameter 'x2', 0.38703056835370409, bounds=[0:1]>),
            ('x3',
             <Parameter 'x3', 0.10838326870063941, bounds=[0:1], expr='1-x1-x2'>)])

In [13]:
result.params

Parameters([('x1', <Parameter 'x1', 0.5045861629456565, bounds=[0:1]>),
            ('x2', <Parameter 'x2', 0.38703056835370409, bounds=[0:1]>),
            ('x3',
             <Parameter 'x3', 0.10838326870063941, bounds=[0:1], expr='1-x1-x2'>)])

In [14]:
result.__dict__

{'aborted': False,
 'aic': -1.3495590347100066,
 'bic': -5.3495590347100066,
 'chisqr': 0.0047502452309016252,
 'errorbars': False,
 'fun': -0.06892202282943838,
 'init_vals': [-0.84843472143810839, 0.12986920118456446],
 'init_values': {'x1': -0.84843472143810839, 'x2': 0.12986920118456446},
 'jac': array([-0.9824626, -0.4559817]),
 'message': 'Warning: Desired error not necessarily achieved due to precision loss.',
 'method': 'Newton-CG',
 'ndata': 1,
 'nfev': 30,
 'nfree': -1,
 'nhev': 0,
 'nit': 2,
 'njev': 27,
 'nvarys': 2,
 'params': Parameters([('x1',
              <Parameter 'x1', 0.5045861629456565, bounds=[0:1]>),
             ('x2', <Parameter 'x2', 0.38703056835370409, bounds=[0:1]>),
             ('x3',
              <Parameter 'x3', 0.10838326870063941, bounds=[0:1], expr='1-x1-x2'>)]),
 'redchi': 0.0047502452309016252,
 'residual': array([-0.06892202]),
 'status': 2,
 'success': False,
 'var_names': ['x1', 'x2'],
 'x': array([ 0.00917245, -0.22790671])}